In [94]:
import pandas as pd
import os

from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score

import nltk
from nltk.corpus import stopwords
from nltk.stem import *


In [54]:
path = "X:/ProgramFiles/JetBrains/PycharmProjects/NBA-predictor/resources/spam-filtering"

filenames = os.listdir(path)
videos = [pd.read_csv(path + '/' + filename, sep=',') for filename in filenames]

dataset = pd.concat(videos)

In [55]:
dataset.sample(frac=1).head(10)

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
183,_2viQ_Qnc69HTQ21lx0wtZ4TFYSpc_gMDApvlQt0q8o,Ishfaq khan,2013-10-21T07:18:41.460000,plz i wilsubscribe me frndzzl subscribe u back,1
426,z12uzjtpanvdhjmtr23asxgyiyjezjxe5,Juan Martinez,2014-07-22T13:22:54.111000,I learned the shuffle because of them﻿,0
12,z122hzpj5v3kgf5zu04chv4xaqalhdr4flw0k,Eduarda Ketrony,2014-08-11T01:43:41,https://www.facebook.com/photo.php?fbid=543627485763966&amp;l=0d878a889c﻿,1
435,z12xhdjrsxm3v550w22oynsjrnmvjhkvj,LuckyMusiqLive,2014-07-21T22:25:54.048000,this song is awesome. these guys are the best. love this video too its hilarious lol. im getting popular fast because i rap with meaning. thumbs up if you piss next to the water in the toilet so it doesnt make noise﻿,1
410,LneaDw26bFs2eQd05L_J9wVmiBlCClqLnM0JUQsB-3Q,YungDreams,NaN,Maybe no one will probably read this. But just in case you do Can You Put A &quot;Thumbs Up &quot;So Others Can See. I Just started rapping seriously (Type in Lunden- 1990 Video) just a simple button? can help my dreams come true. people will ignore this but if you don&#39;t can you listen and subscribe Thank you all God Bless.,1
327,z13wsnj44xfigrbse22iubv5kmebxlebm04,monal shah,2014-11-10T09:00:34,"https://apps.facebook.com/my-polls/utsitcompetition2014?from=user_link&amp;ref_id=ouxg5e . Please open this link and vote for anand niketan international school, the project name is project Fr-e-dom and share it with yours friend.﻿",1
111,z13chhog1nbldb2xp04cgtkxcx35z1djpf0,Free Video Tv,2015-05-21T21:15:41.869000,subscribe to me and I wil subscribe to you back﻿,1
421,z120e5uiiuu4jlaen22dundglo2dypric04,Naga Berapi,2014-07-22T19:53:49.636000,The best song ever!﻿,0
348,z12he50arvrkivl5u04cctawgxzkjfsjcc4,Photo Editor,2015-06-05T14:14:48,hi guys please my android photo editor download. thanks https://play.google.com/store/apps/details?id=com.butalabs.photo.editor﻿,1
401,z12djnnomsvvvpmz204cdh0qgsqdtrq4dfs,mihir Sanjay,2014-09-04T15:24:31.780000,Check out this video on YouTube:﻿,1


In [56]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mateusz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [57]:
stop_words = stopwords.words("english")
vectorizer = CountVectorizer(stop_words=stop_words)

In [58]:
Y = dataset["CLASS"].tolist()

In [59]:
wnl = WordNetLemmatizer()

In [122]:
def hypertune_svm(x, y):
    model = svm.SVC()
    kernel = ['linear', 'poly', 'rbf', 'sigmoid']
    C = [1000, 100, 50, 10, 1.0, 0.1, 0.01]
    gamma = ['scale']
    degree = range(1,11)
    # define grid search
    grid = dict(kernel=kernel,C=C,gamma=gamma, degree=degree)
    
    # model = LogisticRegression()
    # # solvers = ['newton-cg', 'lbfgs', 'liblinear']
    # solvers = ['saga', 'liblinear']
    # penalty = ['l1', 'l2']
    # c_values = [100, 10, 1.0, 0.1, 0.01]
    # # define grid search
    # grid = dict(solver=solvers, penalty=penalty, C=c_values)
    
    cv = KFold(n_splits=10)
    grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='precision', error_score=0)
    grid_result = grid_search.fit(x, y)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [123]:
X_lem = vectorizer.fit_transform(dataset["CONTENT"].apply(wnl.lemmatize))
hypertune_svm(X_lem,Y)

Best: 0.986799 using {'C': 1.0, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}
0.941128 (0.046766) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'linear'}
0.941128 (0.046766) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'poly'}
0.939185 (0.045790) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'rbf'}
0.888437 (0.074375) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}
0.941128 (0.046766) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'}
0.978323 (0.022249) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}
0.939185 (0.045790) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
0.888437 (0.074375) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'sigmoid'}
0.941128 (0.046766) with: {'C': 1000, 'degree': 3, 'gamma': 'scale', 'kernel': 'linear'}
0.977115 (0.027137) with: {'C': 1000, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
0.939185 (0.045790) with: {'C': 1000, '

## Bags of words with stemming

In [73]:
stemmer = PorterStemmer()

In [124]:
X_stem = vectorizer.fit_transform(dataset["CONTENT"].apply(stemmer.stem))
hypertune_svm(X_stem,Y)

Best: 0.986767 using {'C': 1.0, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}
0.942138 (0.047476) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'linear'}
0.942138 (0.047476) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'poly'}
0.940140 (0.045255) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'rbf'}
0.886586 (0.068650) with: {'C': 1000, 'degree': 1, 'gamma': 'scale', 'kernel': 'sigmoid'}
0.942138 (0.047476) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'}
0.978209 (0.022405) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}
0.940140 (0.045255) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
0.886586 (0.068650) with: {'C': 1000, 'degree': 2, 'gamma': 'scale', 'kernel': 'sigmoid'}
0.942138 (0.047476) with: {'C': 1000, 'degree': 3, 'gamma': 'scale', 'kernel': 'linear'}
0.977267 (0.026840) with: {'C': 1000, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
0.940140 (0.045255) with: {'C': 1000, '

In [120]:
model = svm.SVC(kernel='rbf', C=10, gamma='scale', degree=2)
scores_acc = cross_val_score(model, X_lem, Y, cv=10, scoring='accuracy')
scores_prec = cross_val_score(model, X_lem, Y, cv=10, scoring='precision')

In [121]:
scores_acc.mean(), scores_prec.mean()

(0.9488984824699109, 0.950040593728599)